In [51]:
import pandas as pd
import numpy as np
import urllib
import requests
from bs4 import BeautifulSoup
import os
from time import sleep
import time
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
import random 

In [52]:
import spacy
nlp = spacy.load("pt_core_news_sm")

In [72]:
# base_inicial = "links_folha_09_2023.csv"
# base_inicial = "links_estadao_09_2023.csv"
base_inicial = "links_correio_09_2023.csv"

df = pd.read_csv(base_inicial)
df = df.reset_index()
df.columns = ["n","links"]
df.shape


(10, 2)

In [73]:
df.head()

,n,links
0,0,https://www.correiodopovo.com.br/cadernodesaba...
1,1,https://www.correiodopovo.com.br/not%C3%ADcias...
2,2,https://www.correiodopovo.com.br/not%C3%ADcias...
3,3,https://www.correiodopovo.com.br/not%C3%ADcias...
4,4,https://www.correiodopovo.com.br/arteagenda/fi...


In [74]:
def limpa_paragrafos(link, tamanho):
    
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.77 Safari/537.36"}
    try:
        req = Request(link, headers = headers)
        response = urlopen(req)
       
    except HTTPError as e: # erro de segurança do site, pode ser necessário usar user-Agent
        print(e.status, e.reason)
        return ["404_nao_encontrado"]

    except URLError as e:
        print(e.reason)
        return ["404_nao_encontrado"]

    t = random.randint(1, 10)
    # time.sleep(t)
    
#     html = urllib.request.urlopen(link).read()
    html = response.read()
    soup = BeautifulSoup(html)

    for script in soup(["script", "style"]):
        script.decompose()

    lista = []
    for i in soup.stripped_strings:
        lista.append(i)
    
#     print(lista)

    miolo = []
    for i in lista:
        if len(i) > tamanho:
            miolo.append(i.replace('\r\n',''))
#             print(miolo)

    paragrafos = []
    for i in range(len(miolo)):
        if 'esquizo' in miolo[i]:
            paragrafos.append(miolo[i])
    
    t = random.randint(1, 10)
    time.sleep(t)
    
#     for i in range(len(paragrafos)):
#         lista_final = []
#         doc = nlp(paragrafos[i].lower())
#         for token in doc:
#             valido = not token.is_stop and token.is_alpha
# #             valido = token.is_stop
#             if valido:
#                 lista_final.append(token.text)
    
#         limpo = ' '.join(lista_final)
#         paragrafos[i] = limpo
#         time.sleep(1)

    return paragrafos, miolo

In [75]:
link = df.links.values[0]
link

'https://www.correiodopovo.com.br/cadernodesabado/artaud-75-anos-ap%C3%B3s-sua-morte-1.995539'

In [76]:
paragrafos, miolo = limpa_paragrafos(link, 70)

In [77]:
str(paragrafos)

"['A psicanalista Évelyne Grossman contextualiza as reverberações da obra de Artaud: “Durante os anos 60, a partir da publicação, em São Francisco, de uma antologia de seus textos, impôs-se sobre os Estados Unidos sua imagem como um drogado, adepto de todos os delírios, trovador dos paraísos artificiais e da liberação social. Promovido como profeta da Beat Generation, Artaud se funde à revolução hippie e o pop art. Na mesma época na Europa, a luta contra a internação asilar conduzida pelo movimento antipsiquiátrico britânico (Ronald D. Laing, David Cooper), suas hipóteses de uma gênese familiar e social da esquizofrenia registrado nas obras de Michel Foucault ou de Gilles Deleuze sobre a loucura, suscitaram toda uma série de leituras militantes dos textos de Artaud”.']"

In [78]:
str(miolo)

"['Poeta, ator, escritor e dramaturgo francês Antonin Artaud, morto há 75 anos', 'Dia 4 de março de 1948, Antonin Artaud é encontrado caído e morto, perto de sua cama, em Paris. Eis o fim de todos os vivos, quando a moira Átropos corta o fio do destino, como se fechassem as cortinas do palco. Artaud, no entanto, legou-nos uma obra imensa e diversa, até agora e mais do que nunca, centro de investigação de pesquisadores e de artistas. Homem nascido em 1896, teve uma trajetória potente, revolucionária e incandescente em todos os meios onde atuou, refletiu, semeou discórdias e propôs o retorno às origens míticas, contra os artificialismos e as tendências dos bons modos pensantes e artísticos. Revoltou-se contra a sociedade burguesa, totalitária e ocidental, durante e após as duas Guerras Mundiais. Carregou e sustentou toda a sorte de males no corpo, passando a encarnar o “mal-estar” freudiano extremo: suportar as forças da natureza, encarar a decrepitude do corpo e se relacionar com o “out

In [79]:
df.links.iloc[1]

'https://www.correiodopovo.com.br/not%C3%ADcias/cidades/institui%C3%A7%C3%B5es-oferecem-servi%C3%A7os-de-justi%C3%A7a-e-assist%C3%AAncia-social-a-mulheres-em-a%C3%A7%C3%A3o-do-trt4-1.1009475'

In [80]:
n_links = df.shape[0]

In [81]:
for i in range(0, n_links):

    print(f"Processando {i} de {n_links}...")
    
    link = df.links.iloc[i]
    try: 
        paragrafos, miolo = limpa_paragrafos(link, 70)
        df.loc[(df.links == link), "paragrafos_resgatados"] = str(paragrafos)
        df.loc[(df.links == link), "texto_completo"] = str(miolo)
    except:
        df.loc[(df.links == link), "paragrafos_resgatados"] = "[]"
        df.loc[(df.links == link), "texto_completo"] = "[]"

Processando 0 de 10...
Processando 1 de 10...
Processando 2 de 10...
Processando 3 de 10...
Processando 4 de 10...
Processando 5 de 10...
Processando 6 de 10...
Processando 7 de 10...
Processando 8 de 10...
Processando 9 de 10...


In [82]:
df.head()

,n,links,paragrafos_resgatados,texto_completo
0,0,https://www.correiodopovo.com.br/cadernodesaba...,['A psicanalista Évelyne Grossman contextualiz...,"['Poeta, ator, escritor e dramaturgo francês A..."
1,1,https://www.correiodopovo.com.br/not%C3%ADcias...,['A dona de casa Maria Inês Silva dos Santos v...,['Instituições oferecem serviços de justiça e ...
2,2,https://www.correiodopovo.com.br/not%C3%ADcias...,"['Na terça-feira, Alckmin estará em almoço da ...","['No mês passado, a Confederação da Agricultur..."
3,3,https://www.correiodopovo.com.br/not%C3%ADcias...,['A dona Helena Zelmanovitz está entre as milh...,['Atividades chamam a atenção para o Dia Mundi...
4,4,https://www.correiodopovo.com.br/arteagenda/fi...,"['Para encerrar, o brasileiro “Nise: O Coração...",['Conquistas feitas por mulheres e mostradas e...


In [83]:
df1 = df.query("paragrafos_resgatados != '[]'")

In [84]:
df1.shape

(10, 4)

In [85]:
df1.to_csv(f"{base_inicial}_completo_resgatado.csv", index = False)

In [102]:
# df[df.paragrafos_resgatados.notnull()].to_csv("links_estadao_resgatados_parte_3.csv", index = False)
# df[df.paragrafos_resgatados.isnull()].to_csv("links_estadao_nulos.csv", index = False)


In [106]:
# df = pd.read_csv("links_estadao_resgatados.csv")
# df.shape

In [86]:
t = df.query("paragrafos_resgatados == '[]'")
t

,n,links,paragrafos_resgatados,texto_completo


In [70]:
t.shape

(2, 4)

In [71]:
t.to_csv(f"{base_inicial}_resgatado_vazio.csv", index = False)